In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score

# import keras
# import tensorflow as tf
from joblib import dump, load


import importlib
# import neural_networks as nn
# import plotting as pl
import utils

import pyreadr

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)

In [15]:
### import feature name for plotting
# feature_categories_DE = pyreadr.read_r('F:/PROJEKTE/FARM/Daten/Datenanalyse/ML/ml_code/output/useful/colors_feature_categories_DE.RDS')[None]
# feature_categories_DE = pyreadr.read_r('../colors_feature_categories_ENG_update.RDS')[None]


# feature_categories_DE.loc[feature_categories_DE.abbr=='other_nonagri_pu', 'short_name_feature'] = 'Sonst. nlw. Fl. m. pot. Pest.'
# feature_categories_DE.loc[feature_categories_DE.abbr=='other_nonagri_wpu', 'short_name_feature'] = 'Sonst. nlw. Fl. o. pot. Pest.'
# feature_categories_DE.loc[feature_categories_DE.abbr=='other_agri_pu', 'short_name_feature'] = 'Sonst. lw. Fl. m. pot. Pest.'
# feature_categories_DE.loc[feature_categories_DE.abbr=='other_agri_wpu', 'short_name_feature'] = 'Sonst. lw. Fl. o. pot. Pest.'
# feature_categories_DE.loc[feature_categories_DE.abbr=='other', 'short_name_feature'] = 'Sonst. nlw. Fl.'
# # correct encoding
# for col in feature_categories_DE.select_dtypes(['object']).columns:
#     feature_categories_DE[col] = feature_categories_DE[col].str.replace('Ã¤', 'ä')
#     feature_categories_DE[col] = feature_categories_DE[col].str.replace('Ã¼', 'ü')
#     feature_categories_DE[col] = feature_categories_DE[col].str.replace('Ã¶', 'ö')
#     feature_categories_DE[col] = feature_categories_DE[col].str.replace('ÃŸ', 'ß')
# feature_categories_DE.head()
# feature_dict = dict(feature_categories_DE.loc[:, ['feature', 'short_name_feature']].values)
# feature_dict['Makroporen_rounded'] = 'Makroporenklasse'
# feature_dict['arable_land_new'] = 'Ackerland'

In [ ]:
feature_dict

In [17]:
df_red = pyreadr.read_r('../preprocessed_red_features_sentinel.RDS')[None]
df_all = pyreadr.read_r('../preprocessed_red_features_sentinel.RDS')[None]
# replace categorical strings by True/False for each category
dummy_columns = df_red.select_dtypes(['object', 'category']).columns.drop(['lawa_name', 'messstellen_id', 'bundesland', 'messnetz', 'dt_name', 'engl_name', 'conc_group'])
df_red_1hot = pd.get_dummies(df_red, columns=dummy_columns)
dummy_columns = df_all.select_dtypes(['object', 'category']).columns.drop(['lawa_name', 'messstellen_id', 'bundesland', 'messnetz', 'dt_name', 'engl_name', 'conc_group'])
df_all_1hot = pd.get_dummies(df_all, columns=dummy_columns)

In [18]:
prio_compounds = ['desphenyl-chloridazon', 'metazachlor esa', 'metazachlorsäure', 'methyl-desphenylchloridazon', 'metolachlor esa', 'metolachlor-ca', 's-metolachlor-metabolit noa 413173', 'trifluoressigsaeure']
feature_selections = list('AC')

In [19]:
def f1_from_cm(cm):
    recall = np.diag(cm) / np.sum(cm, axis=1)
    precision = np.diag(cm) / np.sum(cm, axis=0)
    f1 = 2 * (precision * recall) / (precision + recall)
    return np.mean(f1)

In [20]:
relevant_sids = {
    'desphenyl-chloridazon': [14, 16], 
    'metazachlor esa': [12, 16, 20], # 23 
    'metazachlorsäure': [12, 16, 20], # 23
    'methyl-desphenylchloridazon': [14, 16], 
    'metolachlor esa': [7, 9, 10, 17, 19], # 27 
    'metolachlor-ca': [7, 9, 10, 17, 19], # 27 
    's-metolachlor-metabolit noa 413173': [7, 9, 10, 17, 19], # 27 
    'trifluoressigsaeure': [1, 2, 3, 4, 7, 9, 10, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21] # 22, 23, 24, 25, 26, 27
}

In [21]:
n_classes = 2
n_folds = 5 # outer folds
n_outer_folds = 5
i_ini = 0
n_ini = 1

### compare performances

In [ ]:
nc = len(prio_compounds)
f, axs = plt.subplots(2, int(nc/2), figsize=(3*nc/2, 6), sharey=True)
# ax.set_xticks()
folder_name='18_2classesMTOcorrection'
n_classes=2
feature_selections=list('AC')
for i, compound_name in enumerate(prio_compounds[:]):
    ax = axs.flatten()[i]
    ax.set_xticks(range(len(feature_selections)), ['reduziert', 'alle'])
    # ax.set_xticks(range(len(feature_selections)), feature_selections)
    ax.set_xlim(-.5, len(feature_selections)-.5)
    ax.set_title(compound_name.title(), fontdict={'fontsize':10})

    for j, feature_selection in enumerate(feature_selections):
        
        output_folder = f'F:/PROJEKTE/FARM/Ephraim_Erkens/bgr-grwv/develoments_BGR/EE/output/nestedCV/{folder_name}/{compound_name}/feature_selection_{feature_selection}/'
        cm_stacked = np.zeros((n_classes, n_classes))

        try:
            performances = pd.read_csv(output_folder + 'performances.csv')
            for fold in range(n_folds):
                cm = pd.read_csv(output_folder + f'data/cm_stacked_outerFold_outerFold{fold}.csv', index_col=0)
                cm_stacked += cm.values
        except FileNotFoundError:
            continue
        f1 = performances[performances.test_set=='outer'].f1_tuned
        mean_f1 = f1.mean()
        ax.scatter(j, mean_f1, marker='s', s=30, label='Durchschnitt')
        ax.scatter([j]*len(f1), f1, color='k', marker='.', label='Outer Folds')
        f1_cm = f1_from_cm(cm_stacked)
        # ax.scatter(j, f1_cm, marker='o', fc='none', ec='k', s=50, label='from cm')

axs[0, 0].set_ylabel('F1-Score')
axs[1, 0].set_ylabel('F1-Score')
# axs[0, 2].set_xlabel('Feature-Auswahl', x=0)
axs[1, 2].set_xlabel('Feature-Auswahl', x=0)
handles, labels = axs[0, 0].get_legend_handles_labels()
axs[0, 0].legend(handles[:2], labels[:2], loc='lower right')

f.subplots_adjust(wspace=0.0, hspace=.3)
for ax in axs.flatten():
    ax.grid(alpha=.2)
    ax.set_axisbelow(True)

f.suptitle('Performance FNN Nested Cross-Validation (2 Klassen)')
f.savefig(f'F:/PROJEKTE/FARM/Ephraim_Erkens/bgr-grwv/develoments_BGR/EE/output/nestedCV/{folder_name}/f1Scores_allCompounds.png', bbox_inches='tight')
plt.show()

In [ ]:
cm.sum(axis=1)

### compare performances (3 classes)

In [ ]:
nc = len(prio_compounds)
f, axs = plt.subplots(2, int(nc/2), figsize=(3*nc/2, 6), sharey=True)
# ax.set_xticks()

folder_name='19_3classesMTOcorrection'
feature_selections = list('AC')
n_classes=3

for i, compound_name in enumerate(prio_compounds[:]):
    ax = axs.flatten()[i]
    # ax.set_xticks(range(len(feature_selections)), feature_selections)
    ax.set_xticks(range(len(feature_selections)), ['reduziert', 'alle'])
    ax.set_xlim(-.5, len(feature_selections)-.5)
    ax.set_title(compound_name.title(), fontdict={'fontsize':10})

    for j, feature_selection in enumerate(feature_selections[:]):
        
        output_folder = f'F:/PROJEKTE/FARM/Ephraim_Erkens/bgr-grwv/develoments_BGR/EE/output/nestedCV/{folder_name}/{compound_name}/feature_selection_{feature_selection}/'
        cm_stacked = np.zeros((n_classes, n_classes))
        
        try:
            performances = pd.read_csv(output_folder + 'performances.csv')
            for fold in range(n_folds):
                cm = pd.read_csv(output_folder + f'data/cm_stacked_outerFold_outerFold{fold}.csv', index_col=0)
                cm_stacked += cm.values
        except FileNotFoundError:
            continue
        f1 = performances[performances.test_set=='outer'].f1_tuned
        mean_f1 = f1.mean()
        ax.scatter(j, mean_f1, marker='s', s=30, label='Durchschnitt')
        ax.scatter([j]*len(f1), f1, color='k', marker='.', label='Outer Folds')
        f1_cm = f1_from_cm(cm_stacked)
        # ax.scatter(j, f1_cm, marker='o', fc='none', ec='k', s=50, label='from cm')
        
        

axs[0, 0].set_ylabel('F1-Score')
axs[1, 0].set_ylabel('F1-Score')
# axs[0, 2].set_xlabel('Feature-Auswahl', x=0)
axs[1, 2].set_xlabel('Feature-Auswahl', x=0)
handles, labels = axs[-1, -1].get_legend_handles_labels()
axs[0, 0].legend(handles[:2], labels[:2], loc='lower right')

f.subplots_adjust(wspace=0.0, hspace=.3)
for ax in axs.flatten():
    ax.grid(alpha=.2)
    ax.set_axisbelow(True)

f.suptitle('Performance FNN Nested Cross-Validation (3 Klassen)')
# f.savefig(f'F:/PROJEKTE/FARM/Ephraim_Erkens/bgr-grwv/develoments_BGR/EE/output/nestedCV/{folder_name}/f1Scores_allCompounds.png', bbox_inches='tight')
plt.show()

In [ ]:
# plot comparison for TFA
f, ax = plt.subplots()

model_names = ['07_cleanOutput']

### plot performance bar plots

In [ ]:
folder_name='19_3classesMTOcorrection'
feature_selections = list('AC')
n_classes=3
for i, compound_name in enumerate(prio_compounds[:]):

    for feature_selection in feature_selections:
        skip=False
        output_folder = f'F:/PROJEKTE/FARM/Ephraim_Erkens/bgr-grwv/develoments_BGR/EE/output/nestedCV/{folder_name}/{compound_name}/feature_selection_{feature_selection}/'
        cm_stacked = np.zeros((n_classes, n_classes))
        for fold in range(n_folds):
            # cm = np.load(output_folder + f'models/cm_nCV_outerFold{fold}.npy')
            try:
                cm = pd.read_csv(output_folder + f'data/cm_stacked_outerFold_outerFold{fold}.csv', index_col=0)
                cm_stacked += cm.values
            except FileNotFoundError:
                skip=True
                continue
        if skip:
            print(compound_name, 'not found')
            continue
        print(compound_name, feature_selection)
        f, ax = plt.subplots(figsize=(3,6), dpi=150)
        pl.plot_classes_stacked(cm_stacked, from_cm=True, ax=ax, show=False, class_boundaries=[0.1, 1])
        f.savefig(output_folder+'fig/performance_barPlotStacked.png', bbox_inches='tight')
        plt.close()
        out_data = pd.DataFrame(cm_stacked, index=[f'{i_class+1}_true' for i_class in range(n_classes)], columns=[f'{i_class+1}_pred' for i_class in range(n_classes)])
        out_data.to_csv(output_folder+'data/cm_stacked.csv')

### plot permutation importances

In [25]:
keys = df_all.columns[~df_all.columns.isin(df_all_1hot.columns)]
values = df_all_1hot.columns[~df_all_1hot.columns.isin(df_all.columns)]
category_dict = {key:[] for key in keys}
for key in keys:
    for value in values:
        if key in value:
            category_dict[key].append(value)
all_catFeatures = [feature for sublist in category_dict.values() for feature in sublist]
inv_category_dict = {}
for k in category_dict.keys():
    for v in category_dict[k]:
        inv_category_dict[v]=k

In [ ]:
# importlib.reload(utils)
importlib.reload(pl)

folder_name = '19_2classesMTOcorrection'
feature_selections = list('AC')
n_classes = 3


n_features_ = 20

shrink_factor=2 # for plot dimensions

for i, compound_name in enumerate(prio_compounds[:]):
    
    for feature_selection in feature_selections:
        print('feature selection:', feature_selection)
        
        output_folder = f'F:/PROJEKTE/FARM/Ephraim_Erkens/bgr-grwv/develoments_BGR/EE/output/nestedCV/{folder_name}/{compound_name}/feature_selection_{feature_selection}/'
 
        try:
            feature_names = list(np.load(f'{output_folder}data/feature_names.npy', allow_pickle=True))
            fi = pd.read_csv(f'{output_folder}data/permutation_importance_{compound_name}.csv') # feature importance
        except FileNotFoundError:
            print(compound_name, 'not found')
            continue
        fi = fi.groupby('lawa_name').mean().reset_index().drop(columns=['outer_fold', 'lawa_name', 'initialization']) # mean of all folds
        fi = fi.T.rename(columns={0:'importance'}) # transpose
        fi['feature'] = fi.index
        # fi = fi.drop('SD_1')
        if n_classes==2:
            fi = fi.drop('threshold')
        elif n_classes==3:
            fi = fi.drop('threshold1')
            fi = fi.drop('threshold2')
            fi = fi.drop('threshold3')
        fi['is_categorical'] = ~fi.feature.isin(df_all.columns)
        fi.loc[fi.is_categorical, 'feature_name'] = fi.feature.map(inv_category_dict)
        fi.loc[~fi.is_categorical, 'feature_name'] = fi.feature
        n_features = n_features_ if len(fi.feature_name.unique())>n_features_ else len(fi.feature_name.unique())
        
        y_min = -1.5/shrink_factor
        y_max = n_features-0.5/shrink_factor
        f, ax = plt.subplots(dpi=100, figsize=(4, n_features/shrink_factor))
        pl.plot_permutation_importance(fi, ax=ax, n_features=n_features)
        ax.set_ylim(y_min, y_max)
        ax.set_title(compound_name)
    
        f.savefig(f'{output_folder}fig/permutationImportance_{compound_name}_{feature_selection}.png', bbox_inches='tight')
        # plt.close()
        print(compound_name)
        # plt.show()
        plt.close()

### Plot partial dependences

In [27]:
# prio_compounds = ['desphenyl-chloridazon', 'metazachlor esa', 'metazachlorsäure', 'methyl-desphenylchloridazon', 
# 'metolachlor esa', 'metolachlor-ca', 's-metolachlor-metabolit noa 413173', 'trifluoressigsaeure']
# prio_compounds = ['metolachlor-ca',]
prio_compounds = ['trifluoressigsaeure']

feature_selections = ['A']

n_outer_folds = 5
grid_resolution = 100

# folder_name = '07_cleanOutput'
# output_folder = f'F:/PROJEKTE/FARM/Ephraim_Erkens/bgr-grwv/develoments_BGR/EE/output/nestedCV/{folder_name}/{compound_name}/feature_selection_{feature_selection}/'

#### 2 classes, single plots

In [28]:
import warnings
warnings.filterwarnings("ignore")

In [29]:
# feature_names = np.load(f'../data/feature_names.npy', allow_pickle=True)
# feature_names

In [30]:
compound_name = prio_compounds[0]

In [31]:
feature_names = np.load(f'../13_paper2classes/{prio_compounds[0]}/feature_selection_A/data/feature_names.npy', allow_pickle=True)
feature_names

array(['filter_ok_unter_gok', 'sand_depth_weighted_mean',
       'corg_gehalt_depth_weighted_mean', 'gwn_mean',
       'area_sentinel_id_1', 'area_sentinel_id_2', 'area_sentinel_id_3',
       'area_sentinel_id_4', 'area_sentinel_id_7', 'area_sentinel_id_9',
       'area_sentinel_id_10', 'area_sentinel_id_12',
       'area_sentinel_id_13', 'area_sentinel_id_14',
       'area_sentinel_id_15', 'area_sentinel_id_16',
       'area_sentinel_id_17', 'area_sentinel_id_18',
       'area_sentinel_id_19', 'area_sentinel_id_20',
       'area_sentinel_id_21', 'Makroporen_rounded_1',
       'Makroporen_rounded_2', 'Makroporen_rounded_3', 'HA_K', 'HA_K/Ka',
       'HA_K/P', 'HA_P', 'kf_bez_1', 'kf_bez_2', 'kf_bez_3'], dtype=object)

In [32]:
# for i_feature, feature in enumerate(feature_names):
#     print(feature)
#     for outer_fold_index in range(5):
#         print(f"Outer fold index {outer_fold_index}")
#         X_train = np.load(f'../13_paper2classes/{compound_name}/feature_selection_A/data/X_outer_train_nCV_outerFold{outer_fold_index}.npy')
#         scaler = load(f'../13_paper2classes/{compound_name}/feature_selection_A/data/StdScaler_outerFold{outer_fold_index}.bin')

#         # ax.plot(x[:, outer_fold_index], 1-y.mean(axis=0)[:, outer_fold_index], 'cornflowerblue', linewidth=.5)

#         dummy_df = pd.DataFrame(columns=feature_names)
#         dummy_df[feature] = X_train[:, i_feature]
#         print(X_train[:, i_feature].shape)
#         x_inv_trans = scaler.inverse_transform(dummy_df)
#         x_dataPoints = x_inv_trans[:, i_feature]

#         # if i_feature == 0:
#         #     # print(dummy_df)
#         #     print(x_inv_trans.shape)
#         #     print(x_dataPoints.shape)
#         #     print(x[:, outer_fold_index].shape)
#         #     print(y.mean(axis=0)[:, outer_fold_index].shape)
#         #     break
#     # break


In [33]:
# x = np.load(f'../13_paper2classes/{compound_name}/feature_selection_A/data/x_Pdp_{compound_name}_{feature.replace('/', '_')}.npy')
# x.shape

In [34]:
# y = np.load(f'../13_paper2classes/{compound_name}/feature_selection_A/data/y_Pdp_{compound_name}_{feature.replace('/', '_')}.npy')
# y.shape

In [35]:
feature_name_dict = {  }

In [ ]:
color = 'cornflowerblue' #'.3'

correct_soc = True
if correct_soc:
    scale = 1.72**2
else:
    scale = 1

for compound_name in prio_compounds:
    # print(compound_name)
    for feature_selection in feature_selections:
        # print(feature_selection)
        output_folder = f'../output/'
        feature_names = np.load(f'../13_paper2classes/{compound_name}/feature_selection_A/data/feature_names.npy', allow_pickle=True)
        # print(feature_names)
        if feature_selection == 'A':
            keys = df_red.columns[~df_red.columns.isin(df_red_1hot.columns)]
            values = df_red_1hot.columns[~df_red_1hot.columns.isin(df_red.columns)]
        if feature_selection == 'C':
            keys = df_all.columns[~df_all.columns.isin(df_all_1hot.columns)]
            values = df_all_1hot.columns[~df_all_1hot.columns.isin(df_all.columns)]
        category_dict = {key:[] for key in keys}
        for key in keys:
            for value in values:
                if key in value:
                    category_dict[key].append(value)
        all_catFeatures = [feature for sublist in category_dict.values() for feature in sublist]
        inv_category_dict = {}
        for k in category_dict.keys():
            for v in category_dict[k]:
                inv_category_dict[v]=k            
        for i_feature, feature in enumerate(feature_names):
            if feature == 'area_sentinel_id_19':
                continue

            if feature != 'corg_gehalt_depth_weighted_mean':
                continue
            # print()
            print(feature + " is being processed.")
            if feature in all_catFeatures:
                is_categorical=True
            else:
                is_categorical=False
        

            if not is_categorical:
                # print("    not categorical")
                # x = np.load(f'../13_paper2classes/{compound_name}/feature_selection_A/data/x_Pdp_{compound_name}_{feature.replace('/', '_')}.npy')
                x = np.load(f"../13_paper2classes/{compound_name}/feature_selection_A/data/x_Pdp_{compound_name}_{(feature.replace('/', '_'))}.npy")
                # y = np.load(f'../13_paper2classes/{compound_name}/feature_selection_A/data/y_Pdp_{compound_name}_{feature.replace('/', '_')}.npy')
                y = np.load(f"../13_paper2classes/{compound_name}/feature_selection_A/data/y_Pdp_{compound_name}_{(feature.replace('/', '_'))}.npy")

                print(x.shape, y.shape)
    
                f, axs = plt.subplots(1, 2, figsize=(5, 4), dpi=200)
                ax = axs[0]
                for outer_fold_index in range(n_outer_folds):

                    X_train = np.load(f'../13_paper2classes/{compound_name}/feature_selection_A/data/X_outer_train_nCV_outerFold{outer_fold_index}.npy')
                    scaler = load(f'../13_paper2classes/{compound_name}/feature_selection_A/data/StdScaler_outerFold{outer_fold_index}.bin')

                    ax.plot(x[:, outer_fold_index]/scale, 1-y.mean(axis=0)[:, outer_fold_index], 'cornflowerblue', linewidth=.5)

                    dummy_df = pd.DataFrame(columns=feature_names)
                    dummy_df[feature] = X_train[:, i_feature]
                    x_inv_trans = scaler.inverse_transform(dummy_df)

                    x_dataPoints = x_inv_trans[:, i_feature]
                    # ax.vlines(x_dataPoints, 0, 0.04, color='blue', alpha=.025,)
                    # axs[1].vlines(x_dataPoints, 0, 0.04, color='blue', alpha=.025)

                ax.plot(x.mean(axis=1)/scale, 1-y.mean(axis=0).mean(axis=1), 'navy', zorder=10)
                
                try:
                    xlabel = feature_dict[feature]
                except KeyError:
                    xlabel = feature
                print('-', xlabel)
                ax.set_xlabel(xlabel)
                ax.set_title('Class 1', fontdict={'fontsize':10})
                # ax.set_ylabel('probability', fontsize=10) # not needed
                    
                ax = axs[1]
                for outer_fold_index in range(n_outer_folds):
                    X_train = np.load(f'../13_paper2classes/{compound_name}/feature_selection_A/data/X_outer_train_nCV_outerFold{outer_fold_index}.npy')
                    ax.plot(x[:, outer_fold_index]/scale, y.mean(axis=0)[:, outer_fold_index], 'cornflowerblue', linewidth=.5, label='Folds')
                    # ax.vlines(X_train[:, i_feature], 0, 0.04, color='green', alpha=.1)
                
                ax.plot(x.mean(axis=1)/scale, y.mean(axis=0).mean(axis=1), 'navy', zorder=10, label='Average')
                ax.set_xlabel(xlabel)
                # ax.set_ylabel('blabla', fontsize=10) # Does not appear good. Overlaps with previous plot.

                ax.set_title('Class 2', fontdict={'fontsize':10})
                handles, labels = ax.get_legend_handles_labels()
                ax.legend(handles[-2:], labels[-2:], frameon=True)

                for ax in axs:
                    ax.set_ylim(0,1)
                    ax.grid(alpha=.3)
                    ax.set_axisbelow(True)
                # f.suptitle(compound_name.title(), y=1)
                # f.savefig(f'../output/Pdp_{compound_name}_{feature.replace('/', '_')}.png', bbox_inches='tight')
                if correct_soc:
                    f.savefig(f"../output_SOC/Pdp_{compound_name}_{(feature.replace('/', '_'))}.png", bbox_inches='tight')
                else:
                    f.savefig(f"../output/Pdp_{compound_name}_{(feature.replace('/', '_'))}.png", bbox_inches='tight')
                plt.close()
                # plt.show()

            if is_categorical:
                print("categorical")
                original_feature = inv_category_dict[feature]
                # print(feature, original_feature)
                cat_feats = category_dict[original_feature]
                n_1hot = len(cat_feats)
                xticks = {cat_feat: i for i, cat_feat in enumerate(cat_feats)}
                if xticks[feature] == 0:
                    f, axs = plt.subplots(1, 2, figsize=(5, 4), dpi=200)

                cat = [c.replace(f'{original_feature}_', '') for c in cat_feats]
                try:
                    xlabel = feature_dict[original_feature]
                except KeyError:
                    # xlabel = original_feature
                    xlabel = feature_dict[original_feature]
                print('-', xlabel)
                for ax in axs:
                    ax.set_xticks(list(xticks.values()), cat)
                    ax.set_xticks(list(xticks.values()), cat)
                    ax.set_xlim(-.5, n_1hot-.5)
                    ax.set_xlabel(xlabel)
                    
                x = np.load(f"../13_paper2classes/{compound_name}/feature_selection_A/data/x_Pdp_{compound_name}_{feature.replace('/', '_')}.npy")
                y = np.load(f"../13_paper2classes/{compound_name}/feature_selection_A/data/y_Pdp_{compound_name}_{feature.replace('/', '_')}.npy")

                axs[0].bar(xticks[feature], 1- y[1, :].mean(), color=color)
                axs[1].bar(xticks[feature], y[1, :].mean(), color=color)

                axs[0].set_title('Class 1', fontdict={'fontsize':10})
                axs[1].set_title('Class 2', fontdict={'fontsize':10})
                
                for ax in axs:
                    ax.set_ylim(0,1)
                    ax.grid(alpha=.3)
                    ax.set_axisbelow(True)

                if xticks[feature] == n_1hot-1:
                    # f.suptitle(compound_name.title(), y=1)
                    f.savefig(f"../output/Pdp_{compound_name}_{original_feature.replace('/', '_')}.png", bbox_inches='tight')
                    
                    plt.close()
                    # plt.show()
print('done')

#### 2 classes, common plot (If you plot the features together!)

In [ ]:
folder_name = '13_paper2classes'
color = 'cornflowerblue' #'.3'
feature_selections = list('A')

for compound_name in prio_compounds[:]:
    print(compound_name)
    if (compound_name in ['metolachlor esa', 'metolachlor-ca', 's-metolachlor-metabolit noa 413173']):
        continue
    for feature_selection in feature_selections[:]:
        count = 0
        print('  ', feature_selection)
        # output_folder = f'F:/PROJEKTE/FARM/Ephraim_Erkens/bgr-grwv/develoments_BGR/EE/output/nestedCV/{folder_name}/{compound_name}/feature_selection_{feature_selection}/'
        feature_names = np.load(f"../{folder_name}/{compound_name}/feature_selection_{feature_selection}/data/feature_names.npy", allow_pickle=True)

        if feature_selection == 'A':
            keys = df_red.columns[~df_red.columns.isin(df_red_1hot.columns)]
            values = df_red_1hot.columns[~df_red_1hot.columns.isin(df_red.columns)]
            n_features = len(df_red.columns.drop(['lawa_name', 'messstellen_id', 'bundesland', 'messnetz', 'dt_name', 'engl_name', 'conc_group', 'x', 'y'])) - df_red.columns.str.contains('area_sentinel').sum() + len(relevant_sids[compound_name])
            
        if feature_selection == 'C':
            keys = df_all.columns[~df_all.columns.isin(df_all_1hot.columns)]
            values = df_all_1hot.columns[~df_all_1hot.columns.isin(df_all.columns)]
            n_features = len(df_all.columns.drop(['lawa_name', 'messstellen_id', 'bundesland', 'messnetz', 'dt_name', 'engl_name', 'conc_group', 'x', 'y', 'arable_land_new'])) - df_all.columns.str.contains('area_sentinel').sum() + len(relevant_sids[compound_name])
            
        category_dict = {key:[] for key in keys}
        for key in keys:
            for value in values:
                if key in value:
                    category_dict[key].append(value)
        all_catFeatures = [feature for sublist in category_dict.values() for feature in sublist]
        inv_category_dict = {}
        for k in category_dict.keys():
            for v in category_dict[k]:
                inv_category_dict[v]=k


        n_cols = 4
        n_rows = math.ceil(n_features/n_cols)
        f, axs = plt.subplots(nrows=n_rows, ncols=n_cols*3, gridspec_kw={'width_ratios':[5, 5, .5]*n_cols}, figsize=(5*(n_cols+.1), 4*n_rows))

        axf = axs.flatten()
        # for i, ax in enumerate(axf):
        #     ax.set_title(i)
        
        for i_feature, feature in enumerate(feature_names):
            # print('    ', feature)
            if feature in all_catFeatures:
                is_categorical=True
            else:
                is_categorical=False


            if not is_categorical:
                x = np.load(f"../13_paper2classes/{compound_name}/data/x_Pdp_{compound_name}_{feature.replace('/', '_')}.npy")
                y = np.load(f"../13_paper2classes/{compound_name}/data/y_Pdp_{compound_name}_{feature.replace('/', '_')}.npy")
    
                ax = axf[count*3]
                for outer_fold_index in range(n_outer_folds):
                    X_train = np.load(f'../data/X_outer_train_nCV_outerFold{outer_fold_index}.npy')
                    scaler = load(f'../data/StdScaler_outerFold{outer_fold_index}.bin')
                    xplot = x[:, outer_fold_index]
                    yplot = 1-y.mean(axis=0)[:, outer_fold_index]
                    # mask = xplot>1e-10
                    # mask[0] = True
                    index_lastData = np.where(xplot>1e-10)[0][-1]
                    ax.plot(xplot[:index_lastData+1], yplot[:index_lastData+1], '.6', linewidth=.5)
                    dummy_df = pd.DataFrame(columns=feature_names)
                    dummy_df[feature] = X_train[:, i_feature]
                    x_inv_trans = scaler.inverse_transform(dummy_df)
                    x_dataPoints = x_inv_trans[:, i_feature]
                    ax.vlines(x_dataPoints, 0, 0.04, color='k', alpha=.025)
                    axf[count*3+1].vlines(x_dataPoints, 0, 0.04, color='k', alpha=.025)
                ax.plot(x.mean(axis=1)[:index_lastData+1], 1-y.mean(axis=0).mean(axis=1)[:index_lastData+1], color, zorder=10)
                try:
                    xlabel = feature_dict[feature]
                except KeyError:
                    xlabel = feature
                axf[count*3].set_xlabel(xlabel)
                axf[count*3+1].set_xlabel(xlabel)
                ax.set_title('Klasse 1', fontdict={'fontsize':10})
                ax.set_ylabel('Vorhergesagte Wahrscheinlichkeit')
                    
                ax = axf[count*3+1]
                for outer_fold_index in range(n_outer_folds):
                    xplot = x[:, outer_fold_index]
                    yplot = y.mean(axis=0)[:, outer_fold_index]
                    index_lastData = np.where(xplot>1e-10)[0][-1]
                    ax.plot(xplot[:index_lastData+1], yplot[:index_lastData+1], '.6', linewidth=.5, label='Folds')
                    ax.vlines(X_train[:, i_feature], 0, 0.04, color='k', alpha=.1)
                ax.plot(x.mean(axis=1)[:index_lastData+1], y.mean(axis=0).mean(axis=1)[:index_lastData+1], color, zorder=10, label='Durchschnitt')
                ax.set_xlabel(xlabel)
                ax.set_title('Klasse 2', fontdict={'fontsize':10})

                handles, labels = ax.get_legend_handles_labels()

                count += 1

            if is_categorical:
                original_feature = inv_category_dict[feature]
                # print(feature, original_feature)
                cat_feats = category_dict[original_feature]
                n_1hot = len(cat_feats)
                xticks = {cat_feat: i for i, cat_feat in enumerate(cat_feats)}

                x = np.load(f"{output_folder}data/x_Pdp_{compound_name}_{feature.replace('/', '_')}.npy")
                y = np.load(f"{output_folder}data/y_Pdp_{compound_name}_{feature.replace('/', '_')}.npy")

                cat = [c.replace(f'{original_feature}_', '') for c in cat_feats]
                try:
                    xlabel = feature_dict[original_feature]
                except KeyError:
                    xlabel = original_feature
                    
                ax = axf[count*3]
                ax.set_xlabel(xlabel)
                ax.set_xticks(list(xticks.values()), cat)
                ax.set_xticks(list(xticks.values()), cat)
                ax.set_xlim(-.5, n_1hot-.5)
                ax.set_xlabel(xlabel)
                ax.bar(xticks[feature], 1- y[1, :].mean(), color=color)
                ax.set_title('Klasse 1', fontdict={'fontsize':10})
                ax.set_ylabel('Vorhergesagte Wahrscheinlichkeit')
                
                
                ax = axf[count*3+1]
                ax.set_xlabel(xlabel)
                ax.set_xticks(list(xticks.values()), cat)
                ax.set_xticks(list(xticks.values()), cat)
                ax.set_xlim(-.5, n_1hot-.5)
                ax.set_xlabel(xlabel)
                ax.bar(xticks[feature], y[1, :].mean(), color=color)
                ax.set_title('Klasse 2', fontdict={'fontsize':10})
                
                
                if xticks[feature] == n_1hot-1:
                    count += 1

        ax.legend(handles[-2:], labels[-2:], frameon=True, bbox_to_anchor=(1.2, 1), loc='upper left')

        for i, ax in enumerate(axf):
            ax.set_ylim(0,1)
            ax.grid(alpha=.3)
            ax.set_axisbelow(True)
            if ((i+1)%3==0) | (i>=n_features*3):
                f.delaxes(ax)
        f.subplots_adjust(wspace=.4, hspace=.35)
        f.suptitle(compound_name.title(), y=.9) if feature_selection=='C' else f.suptitle(compound_name.title(), y=.95)
        f.savefig(f"{output_folder}fig/pdp/Pdp_{compound_name}.png', bbox_inches='tight")   
        plt.close()